In [3]:
import arviz as az
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.lines import Line2D
style.use('plotting.mplstyle')
import numpy as np
import json
from scipy.stats import gaussian_kde
import sys

sys.path.append('./../code/emceeCode')
from posterior_helper_functions import *

In [ ]:
# Define function to draw n chi-effective values corresponding to a component spin + mass ratio 
# distribution w/ parameters {mu_chi, sigma_chi, MF_cost, sigma_cost, f_spike, sigma_spike, cost_min, Bq}

def drawChieffs(mu_chi, sigma_chi, MF_cost, sigma_cost, f_spike, sigma_spike, cost_min, Bq, n=1): 
    
    # draw uniform component spins + masses
    nRandomDraws = 10000
    samp_idxs = np.arange(nRandomDraws)
    chi1s = np.random.rand(nRandomDraws)
    chi2s = np.random.rand(nRandomDraws)
    cost1s = np.random.rand(nRandomDraws)*2 - 1
    cost2s = np.random.rand(nRandomDraws)*2 - 1
    mAs = np.random.rand(nRandomDraws)*100
    mBs = np.random.rand(nRandomDraws)*100
    m1s = np.maximum(mAs, mBs)
    m2s = np.minimum(mAs, mBs)
    
    # calculate p(spins,masses) for these uniform samples,
    # using functions from posterior_helper_functions.py
    p_chi1 = betaDistributionPlusSpike(chi1s, mu_chi, sigma_chi, f_spike, sigma_spike)
    p_chi2 = betaDistributionPlusSpike(chi2s, mu_chi, sigma_chi, f_spike, sigma_spike)
    p_cost1 = calculate_Gaussian_Mixture_1D(cost1s, 1, sigma_cost, MF_cost, cost_min, 1)
    p_cost2 = calculate_Gaussian_Mixture_1D(cost2s, 1, sigma_cost, MF_cost, cost_min, 1)
    p_masses = p_astro_masses(m1s, m2s, bq=Bq)
    
    weights = p_chi1*p_chi2*p_cost1*p_cost2*p_masses
    weights_normed = weights/np.sum(weights)
    weights_normed[np.where(weights_normed<0)] = 0 # get rid of tiny division errors
    
    # select a subset of the samples subject to the weights
    # calculated from p(spins,masses)
    idxs = np.random.choice(samp_idxs, p=weights_normed, size=n)  
    
    # calculate chi-eff for these samples
    q = m2s[idxs]/m1s[idxs]
    chi_eff = (chi1s[idxs]*cost1s[idxs] + q*chi2s[idxs]*cost2s[idxs])/(1+q)
    
    return chi_eff